In [ ]:
import tweepy
import requests as r
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import csv
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from twitter_keys import Access_Token, Access_Token_Secret, Consumer_Key, Consumer_API_Secret

In [ ]:
auth = tweepy.OAuthHandler(Consumer_Key, Consumer_API_Secret)
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
data_file = './data/twitter_log.csv'

In [ ]:
def find_last():
    last_tweet = api.search('@enveraai')

    last_tweet_text = last_tweet['statuses'][0]['text'].lower()
    last_tweet_id = last_tweet['statuses'][0]['id']
    last_tweet_screen_name = last_tweet['statuses'][0]['user']['screen_name']

    twitter_search_name = re.findall(r"(@[a-z0-9\S]+)", last_tweet_text)
    last_tweet_analysis = twitter_search_name[1]

    return last_tweet_text, last_tweet_id, last_tweet_screen_name, last_tweet_analysis

#Possible problem is when last_tweet is empty.
#In this case, the script should recheck in 5 min and keeping checking unitl last_tweet 
#returns a non-zero value.

In [ ]:
#Possible solution to issue noted in find_last

while(True):
    try:
        find_last()
    except:
        print('No tweets found.')
    
    time.sleep(10)    

In [ ]:
def check_dupilicates(data_file, last_tweet_analysis):
    df = pd.read_csv(data_file)
    last_tweet_names = df['screen_name_analysis'].tolist().unique()

    if last_tweet_analysis in last_tweet_names:
        result = True
    else:
        result = False
    
    return result

#Question.  How do I need to structure the functions to work if I will be using
#input values from other functions?

# Question.  How does the return statement work?  Will the return statements work as written?
# When I ran a smiliar code code block, I got an indentation error on line 6.

In [ ]:
def add_row_extract(last_tweet_text, last_tweet_id, last_tweet_screen_name, last_tweet_analysis):
    with open(data_file, mode = 'a') as f:
        writer = csv.writer(f)
        writer.writerow([last_tweet_text, last_tweet_id, last_tweet_screen_name, last_tweet_analysis])
    
    for x in range(25):
        public_tweets = api.user_timeline(twitter_search_name[1], page = x)
        tweet_text = [tweet['text'] for tweet in public_tweets]

    return tweet_text

#Question.  How do I need to structure the functions to work if I will be using
#input values from other functions?

#Question.  How do I add a header to a newly-created CSV file?

In [ ]:
def analyze_tweets(tweet_text):
    compound_score = []
    analyzer = SentimentIntensityAnalyzer()
    for text_ in tweet_text:
        score = analyzer.polarity_scores(text_)['compound']
        compound_score.append(score)
    return compound_score

#Question.  How do I need to structure the functions to work if I will be using
#input values from other functions?

In [ ]:
def plot_results(compound_score):
    plt.figure(figsize = (10, 10))
    plt.plot(np.arange(1, len(compound_score) + 1), compound_score, marker='o', alpha = 0.5, label = twitter_search_name[1])
    plt.xlabel('Tweets Ago', fontsize = 14)
    plt.ylabel('Compound Score (unitless)', fontsize = 14)
    plt.title('Tweet Sentiment Over Last 500 Tweets', fontsize = 14)
    plt.ylim(-1, 1, 0.25)
    plt.legend(title="Tweets")
    plt.savefig('tweet_sentiment.png')
    plt.show()
    
#Question.  How do I need to structure the functions to work if I will be using
#input values from other functions?

In [ ]:
def app():
    find_last()
    check_dupilicates()
    
    if result == False:
        add_row()
        extract()
        analyze_tweets()
        plot_results()
    else:
        print('Sorry, cannot analyze twitter name')

In [ ]:
#     status_text = 'Thanks for the tweet ' + '@' + last_tweet_screen_name + ' heres the analysis of ' + twitter_search_name[1]
#     api.update_with_media('/Users/gta/dev/hw-7/tweet_sentiment.png', status = status_text, in_reply_to_status_id = last_tweet_id)